# Funkcionalno programiranje

Amer Hasanović

## `Union` uzorak

Format za uzorak je:

```fsharp
Constructor (pat1, pat2, ..., patn)
```

Gdje je:

*  `Constructor` - neki od konstruktora unije sa čijom vrijednosti će se vršiti podudaranje.
* `pat1`, `pat2`, ... `patn` - uzorak

In [2]:
let foo  = function 
  | Some a -> printfn "Imate rezultat: %A" a
  | None -> printfn "Nažalost rezultata nema."

Some 4 |> foo
None |> foo

Imate rezultat: 4
Nažalost rezultata nema.


Alterativno, ukoliko polja konstruktora imaju imena, format može biti:

```fsharp
Constructor (name1 = pat1; name2 = pat2; ..., namen = patn)
```

Gdje je:

*  `Constructor` - neki od konstruktora unije sa čijom vrijednosti će se vršiti podudaranje.
* `pat1`, `pat2`, ..., `patn` - uzorak
* `name1`, `name2`, ..., `namen` - ime polja

In [3]:
type Oblik =
  | Pravougaonik of visina: int * sirina : int 
  | Krug of precnik : int 

let sOblik = function 
  | Pravougaonik (sirina = s; visina = v) 
    -> sprintf "[%d, %d]" v s
  | Krug r 
    -> sprintf "(%d)" r

Pravougaonik (visina = 25, sirina = 8) 
   |> sOblik 
   |> printfn "%s"

Krug 10 |> sOblik |> printfn "%s" 

[25, 8]
(10)


## Polimorfne funkcije

Funkcija može biti parametrizirana i tipovima, tzv **tipske varijable** (type varijable), i u tom slučaju kažemo da je funkcija polimorfna. 

Tipska varijabla: 

* reprezentira svaki validni `F#` tip,
* navodi se u listi tipskih parametara nakon imena funkcije unutar `< >`

Ime tipske varijable počinje sa simbolom **'**,

Eksplicitni format:

```fsharp
let fname<'t1,'t2,...,'tn,'tm> (pat1:'t1) (pat2:'t2) ... (patn:'tn) : 'tm= expr
```

Implicitni format:

```fsharp
let fname (pat1:'t1) (pat2:'t2) ... (patn:'tn) : 'tm = expr
```




Kompajler će u procesu zaključivanja tipova pokušati generalizirani tip kao tipsku varijablu, ukoliko konstatuje da je to moguće.

* Ova generalizacija se vrši ukoliko kompajler svjedoči da će funkcija moći raditi isto **za svaki** tip na mjestu tipske varijable ( tzv **parametric polymorphism** )

In [4]:
let foo<'a, 'b> (a : 'a) (b : 'b) : 'b = b

let bar (a : 'a) (b : 'b) : 'b = b

let tar _ b = b


`F#` funkcija koja je polimorfna može uzeti **samo** monomorfnu funkciju kao parametar (tzv. **Rank-1 ili predicative polymorphism** )

In [5]:
let foo f x y = (f x, f y)

In [6]:
foo id 'a' 'b' |> printfn "%A"
foo id 3.5 -4.3 |> printfn "%A"

('a', 'b')
(3.5, -4.3)


In [7]:
foo id 'a' 5 |> printfn "%A"

Error: input.fsx (1,12)-(1,13) typecheck error This expression was expected to have type
    'char'    
but here has type
    'int'    

## Polimorfni tipovi

`F#` algebarski tip može biti polimorfan ukoliko na mjestu tipa makar jednog od polja u tom tipu figurira tipska varijabla.

Tipska varijabla: 

* reprezentira svaki validni `F#` tip,
* navodi se u listi tipskih parametara kojim je parametriziran polimorfni tip unutar `< >`, i to nakon imena tipa.

Ime tipske varijable počinje sa simbolom **'**,

In [8]:
type Par<'a, 'b> = { first : 'a; second : 'b }

let t = { first = 10; second = "foobar"}
let p = { first = 2.5; second = true }

In [9]:
type Rezultat<'a, 'b> = 
  | Uspjeh of 'a
  | Greska of 'b

let p = Uspjeh (2, 3)
let t : Rezultat<int, string> = Uspjeh 5
let k = Greska "Pokusajte ponovo"

Tipska varijabla reprezentira uvijek monomorfni tip.

* `F#` ne pruža način da polimorfni tip prilikom instanciranja uzme polimorfni tip na mjestu tipske varijable tzv **Rank-1 kind**.

In [10]:
type Foobar = Rezultat<int, Option<double>>

In [11]:
type Tarbar = Rezultat<int, Option>

Error: input.fsx (1,29)-(1,35) typecheck error The type 'Microsoft.FSharp.Core.Option<_>' expects 1 type argument(s) but is given 0
input.fsx (1,29)-(1,35) typecheck error The type 'Microsoft.FSharp.Core.Option<_>' expects 1 type argument(s) but is given 0

## Rekurzivni tipovi

Prilikom definicije tipa koji je algebarska unija `F#` omogućava upotrebu tog tipa na mjestu tipa nekog od polja njegovih članova. 

* Za takav tip kažemo da je **rekurzivan**.

In [12]:
type Lista<'a> =
  | Kraj 
  | Element of 'a * Lista<'a>

In [13]:
let t = Element (2, Element( 1, Element( 4, Element(8, Kraj))))

t |> printfn "%A"

Element (2, Element (1, Element (4, Element (8, Kraj))))


In [14]:
let pushFront e l = Element (e, l)

let (@@) = pushFront

let t' : Lista<int> = 2 @@ 1 @@ 4 @@ 8 @@ Kraj

let t'' : List<int> = 2 :: 1 :: 4 :: 8 :: []

let t''' : List<int> = [2; 1; 4; 8]

In [15]:
t' |> printfn "%A"
t'' |> printfn "%A"

Element (2, Element (1, Element (4, Element (8, Kraj))))
[2; 1; 4; 8]


```fsharp
isEmpty : Lista<'a> -> bool
```

In [16]:
let isEmpty = function 
  | Kraj -> true
  | _ -> false

```fsharp
head : Lista<'a> -> 'a
tail : Lista<'a> -> Lista<'a>
```

In [17]:
let head = function
  | Element(e, xs) -> e
  | _ -> raise (System.ArgumentException "The input list was empty")

let tail = function
  | Element(e, xs) -> xs
  | _ -> raise (System.ArgumentException "The input list was empty")

```fsharp
head' : Lista<'a> -> Option<'a>
tail' : Lista<'a> -> Option< Lista<'a> >
```

In [18]:
let head' = function 
  | Element(e, xs) -> Some e
  | Kraj -> None

let tail' = function 
  | Element(e, xs) -> Some xs
  | Kraj -> None

```fsharp
map : ('a -> 'b) -> Lista<'a> -> Lista<'b>
```

In [19]:
let rec map f l =
  match l with 
  | Kraj -> Kraj
  | Element (e, xs) -> Element ( f e, map f xs )

```fsharp
foldl : ('a -> 'e -> 'a) -> 'a -> Lista<'e> -> 'a 
```

In [20]:
let rec foldl f a l = 
  match l with 
  | Kraj -> a
  | Element(e, xs) -> foldl f (f a e) xs

```fsharp
length : Lista<'a> -> int
suma : Lista<int> -> int
proizvod : Lista <int> -> int
```

In [21]:
let length l = foldl (fun a _ -> a + 1) 0 l
let suma l = foldl (+) 0 l
let proizvod l = foldl (*) 1 l

```fsharp
toStr : Lista<'a> -> string
```

In [22]:
let toStr l = 
   l |> map ( fun e -> sprintf "%A " e ) 
     |> foldl ( fun a e -> a + e ) ""
     |> ( fun s -> "[ " + s + "]")


```fsharp
reverse : Lista<'a> -> Lista<'a>
```

In [23]:
let reverse l =
  let rec loop a = function 
    | Kraj -> a
    | Element(e, xs) -> loop (Element (e, a)) xs
  loop Kraj l



In [24]:
let reverse' l = foldl ( fun a e -> Element (e, a) ) Kraj l 

reverse t = reverse' t

True

```fsharp
(+++) : Lista<'a> -> Lista<'a> -> Lista<'a>
```

In [25]:
let (+++) l1 l2 = 
  foldl ( fun a e -> Element (e, a) ) l2 (reverse l1)

let tt = t +++ t


```fsharp
zipWith : ('a -> 'b -> 'c) -> Lista<'a> -> Lista<'b> -> Lista<'c>
zip : Lista<'a> -> Lista<'b> -> Lista<'a*'b>
```

In [26]:
let rec zipWith f l1 l2 = 
  match (l1, l2) with
  | (Kraj, _) 
    -> Kraj
  | (_, Kraj) 
    -> Kraj
  | ( Element (a1, xs1), Element (a2, xs2) ) 
    -> Element ( f a1 a2, zipWith f xs1 xs2 )

let zip l1 l2 = zipWith ( fun e1 e2 -> (e1, e2) ) l2 l2

let t1 = zip t t 

```fsharp
drop : int -> Lista<'a> -> Lista<'a>
```

In [27]:
let rec drop n foo = 
   match (n, foo) with 
   | (0, xs) -> xs
   | (_, Kraj) -> Kraj
   | (n', Element (_, xs)) -> drop (n'-1) xs

```fsharp
take : int -> Lista<'a> -> Lista<'a>
```

In [28]:
let rec take n l = 
  let rec loop a n' l' =
     match (n', l') with 
     | (0, xs) -> a
     | (_, Kraj) -> Kraj
     | (n'', Element (e, xs)) -> loop (Element (e, a)) (n''-1) xs
  loop Kraj n l |> reverse
